In [14]:
import chimeric_tools as ct

In [15]:
from chimeric_tools import Spatial
from chimeric_tools import Time
print(Time.todayEpiWeek())


12
13


In [17]:
print(Spatial.fromState2FIPS("PA"))

42


In [18]:
print(Spatial.GetStateAbbreviation("Pennsylvania"))

PA


In [16]:
print(Time.fromDates2Epiweeks(["2020-01-22","2020-02-23","2020-03-24"]))

[4, 9, 13]


In [ ]:
import covidcast
from datetime import date
data_7d = covidcast.signal("jhu-csse", "confirmed_7dav_incidence_num", date(2020, 1, 1), date(2022, 6,8),
                        "state", "pa")

from datetime import date
from matplotlib import pyplot as plt
import pandas as pd
# data = data.reset_index()
plt.plot(data_7d.time_value, data_7d.value, label="confirmed_incidence_num")
data_7d.to_csv("covid_pa_confirmed_data_7d.csv")

In [ ]:



import covidcast
from datetime import date
abbrevation_states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
lower_abbrevation_states = [state.lower() for state in abbrevation_states]

for state in lower_abbrevation_states:
    data = covidcast.signal("jhu-csse", "confirmed_7dav_incidence_num", date(2020, 1, 1), date(2022, 6,8),
                            "state", state)
    data.to_csv(f"../data/covid/confirmed_7day/state/{state}_2020_2022.csv")


In [3]:
import covidcast
from datetime import date
data = covidcast.signal("jhu-csse", "confirmed_incidence_num", date(2020, 1, 1), date(2022, 6,),
                        "state", "pa")

data

,geo_value,signal,time_value,issue,lag,missing_value,missing_stderr,missing_sample_size,value,stderr,sample_size,geo_type,data_source
0,pa,confirmed_incidence_num,2022-05-01,2022-05-02,1,0,5,5,1448.0,None,None,state,jhu-csse
0,pa,confirmed_incidence_num,2022-05-02,2022-05-03,1,0,5,5,1466.0,None,None,state,jhu-csse
0,pa,confirmed_incidence_num,2022-05-03,2022-05-04,1,0,5,5,2134.0,None,None,state,jhu-csse
0,pa,confirmed_incidence_num,2022-05-04,2022-05-05,1,0,5,5,2615.0,None,None,state,jhu-csse
0,pa,confirmed_incidence_num,2022-05-05,2022-05-06,1,0,5,5,757.0,None,None,state,jhu-csse
0,pa,confirmed_incidence_num,2022-05-06,2022-05-07,1,0,5,5,388.0,None,None,state,jhu-csse
0,pa,confirmed_incidence_num,2022-05-07,2022-05-08,1,0,5,5,0.0,None,None,state,jhu-csse


In [5]:
data.to_csv("pa_covid_confirmed.csv")